<a href="https://colab.research.google.com/github/Naghme98/AI-course_projects/blob/main/translation_with_prompt_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install \
    transformers \
    datasets \
    evaluate \
    rouge_score\
    loralib \
    bitsandbytes \
    peft --quiet

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 13.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from google.colab import drive
from datasets import load_dataset
from transformers import AutoModelForCausalLM, BitsAndBytesConfig , AutoTokenizer
from transformers import GenerationConfig
from google.colab import userdata
import os
import torch
import evaluate


In [ ]:

drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
fnames = os.listdir("/content/drive/MyDrive/Colab Notebooks")
print(fnames)

['translator-summerizer-task1.ipynb', 'test-Sheet1.csv', 'translation.ipynb', '06_NMT_with_Gemma2_9b.ipynb', 'Untitled', 'translation_with_prompttuning.ipynb']


In [ ]:
df_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/test-Sheet1.csv")
print(df_test.loc[1])

English    She is reading a very interesting book.
Persian       او دارد یک کتاب بسیار جالب می‌خواند.
Name: 1, dtype: object


In [ ]:
model_name = "google/gemma-3-1b-it"
# token = userdata.get('hugging_face_token')
# model_name = "google/gemma-2-9b-it"

# bnbConfig = BitsAndBytesConfig(
#     load_in_4bit = True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16,
# )
tokenizer = AutoTokenizer.from_pretrained( model_name,
                                           use_fast=True,
                                          )
foundational_model = AutoModelForCausalLM.from_pretrained( model_name,
                                              device_map="cuda",
                                              # quantization_config=bnbConfig,
                                              torch_dtype=torch.bfloat16)

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/899 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

## Preparing the training dataset

In [ ]:
huggingface_dataset_name = "shenasa/English-Persian-Parallel-Dataset"
dataset = load_dataset(huggingface_dataset_name)

In [ ]:
dataset = dataset['train']
dataset

In [ ]:
import pandas as pd
from datasets import Dataset


source_lang_col = 'src_lang'
target_lang_col = 'tgt_lang'

df_train = dataset.to_pandas()
df_train.columns = ['src_lang', 'tgt_lang']

dataset = Dataset.from_pandas(df_train)
dataset

In [ ]:
dataset = dataset.filter(lambda x: isinstance(x[target_lang_col], str))
dataset = dataset.filter(lambda x: isinstance(x[source_lang_col], str))
dataset

In [ ]:
def preprocess_fn(examples):
    prompts = [
        f"Translate the following English text to Persian:\n"
        f"English: {src}\n\nPersian translation:"
        for src in examples[source_lang_col]
    ]
    targets = examples[target_lang_col]
    full_texts = [p + " " + t for p, t in zip(prompts, targets)]

    model_inputs = tokenizer(
        full_texts,
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )

    labels = model_inputs["input_ids"].clone()

    for i, prompt in enumerate(prompts):
        prompt_ids = tokenizer(prompt, truncation=True, max_length=128)["input_ids"]
        prompt_len = len(prompt_ids)
        labels[i][:prompt_len] = -100  # mask prompt from loss

    model_inputs["labels"] = labels
    # check this. i still have questions about it.
    display(model_inputs)
    return model_inputs


In [ ]:
tokenized_train = dataset.map(
    preprocess_fn,
    batched=True,
    remove_columns=[source_lang_col, target_lang_col]
)

In [ ]:
split_dataset = tokenized_train.train_test_split(test_size=0.2, seed=42)
tokenized_train = split_dataset["train"]
tokenized_eval = split_dataset["test"]

## Peft - prompt tunning

In [ ]:
NUM_VIRTUAL_TOKENS = 4
NUM_EPOCHS = 6

In [ ]:
from peft import  get_peft_model, PromptTuningConfig, TaskType, PromptTuningInit

generation_config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM, #This type indicates the model will generate text.
    prompt_tuning_init=PromptTuningInit.RANDOM,  #The added virtual tokens are initializad with random numbers
    num_virtual_tokens=NUM_VIRTUAL_TOKENS, #Number of virtual tokens to be added and trained.
    tokenizer_name_or_path=model_name #The pre-trained model.
)

In [12]:
peft_model_prompt = get_peft_model(foundational_model, generation_config)
print(peft_model_prompt.print_trainable_parameters())

NameError: name 'foundational_model' is not defined

In [ ]:
from transformers import TrainingArguments
def create_training_arguments(path, learning_rate=0.0035, epochs=6):
    training_args = TrainingArguments(
        output_dir=path, # Where the model predictions and checkpoints will be written
        use_cpu=False,
        auto_find_batch_size=True, # Find a suitable batch size that will fit into memory automatically
        learning_rate= learning_rate, # Higher learning rate than full Fine-Tuning
        num_train_epochs=epochs,
        report_to="none"

    )
    return training_args

In [ ]:
import os

working_dir = "./"

#Is best to store the models in separate folders.
#Create the name of the directories where to store the models.
output_directory_prompt =  os.path.join(working_dir, "peft_outputs_prompt")

#Just creating the directoris if not exist.
if not os.path.exists(working_dir):
    os.mkdir(working_dir)
if not os.path.exists(output_directory_prompt):
    os.mkdir(output_directory_prompt)

In [ ]:
training_args_prompt = create_training_arguments(output_directory_prompt, 0.003, NUM_EPOCHS)

In [ ]:
from transformers import Trainer, DataCollatorForLanguageModeling
def create_trainer(model, training_args, train_dataset):
    trainer = Trainer(
        model=model, # We pass in the PEFT version of the foundation model, bloomz-560M
        args=training_args, #The args for the training.
        train_dataset=train_dataset, #The dataset used to train the model.
        data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False) # mlm=False indicates not to use masked language modeling
    )
    return trainer

In [ ]:
#Training first model.
trainer_prompt = create_trainer(peft_model_prompt, training_args_prompt, tokenized_train)
trainer_prompt.train()

In [ ]:
trainer_prompt.model.save_pretrained(output_directory_prompt)

## Bleu score

In [ ]:
from tqdm.notebook import tqdm

bleu = evaluate.load("bleu")

predictions = []
references = []
source_texts_list = []

MAX_LENGTH = 512

with torch.no_grad():
    for i in tqdm(range(len(source_texts)), desc="Processing translations"):
        source_text = source_texts[i]
        reference_text = reference_texts[i]

        prompt = f"""
        Translate the following English text to Persian:
        English: {source_text}

        Persian translation:"""

        inputs = tokenizer(
            prompt,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=MAX_LENGTH
        ).to(device)

        input_token_length = inputs["input_ids"].shape[1]

        full_output_ids = model.generate(
            inputs["input_ids"],
            max_new_tokens=100,
            do_sample=False  # deterministic
        )[0]

        output_only_ids = full_output_ids[input_token_length:]
        cleaned_output = tokenizer.decode(
            output_only_ids,
            skip_special_tokens=True
        )
        prediction_text = cleaned_output.strip()

        predictions.append(prediction_text)
        references.append([reference_text])
        source_texts_list.append(source_text)

results = bleu.compute(predictions=predictions, references=references)

print(f"BLEU Score: {results['bleu']:.4f}")

Corpus BLEU: 0.23873684811562002


In [ ]:
from googletrans import Translator
translator = Translator()
translated = await translator.translate("Hello, Azizam How are you?", dest='fa')
print(translated.text)

/usr/lib/python3.12/importlib/__init__.py:90: RuntimeWarning: coroutine 'Translator.translate' was never awaited
  return _bootstrap._gcd_import(name[level:], package, level)


سلام عزیزم خوبی؟


In [ ]:
from googletrans import Translator
import asyncio

translator = Translator()

async def translate_bulk():
    results = []
    for sentence in df_test["English"].tolist():   # convert Series → Python list
        tr = await translator.translate(sentence, dest='fa')
        results.append(tr.text.split())
    return results

result_google_translate =  await translate_bulk()

In [ ]:
score = corpus_bleu(references, result_google_translate)
print("Corpus BLEU:", score)

Corpus BLEU: 0.36664698779008076
